In [ ]:
import numpy as np
import os.path as op
from pprint import pformat

# EEG utilities
import mne
from mne.preprocessing import ICA, create_eog_epochs
from pyprep.prep_pipeline import PrepPipeline
from autoreject import get_rejection_threshold, validation_curve

# BIDS utilities
from mne_bids import BIDSPath, read_raw_bids
from util.io.bids import DataSink
from bids import BIDSLayout

In [ ]:
# Constants
BIDS_ROOT = '../data/bids'
DERIV_ROOT = op.join(BIDS_ROOT, 'derivatives')
LOWPASS = 300
FS = 2000
REJECT_THRES = 5e-7 # 50 microvolts

# Parse BIDS directory
layout = BIDSLayout(BIDS_ROOT)
subjects = layout.get_subjects()
tasks = layout.get_tasks()
runs = layout.get_runs()
print(subjects, tasks, runs)

## Wrapper
Since we have to loop over all the data files the section below will contain the for loop and all subsequent sections the functions for each preprocessing step.

In [ ]:
for sub_idx, sub in enumerate(subjects)
    sub = subjects[sub_idx]
    task = tasks[sub_idx]
    run = str(run[sub_idx])
    
    # Import data
    bids_path = get_bids_path(sub, task, run)
    raw = import_bids_data(bids_path)
    events, event_ids = read_events(raw)
    raw = set_electrode_positions(raw, 'standard_1020', 'Aux1')
    
#     Make copy of unprocessed raw data for later comparison
#     raw_unprocessed = raw.copy()
    
    # Resampling and PREP
    raw, events = resample(FS, events)
    raw = run_PREP(raw, sub_idx, LOWPASS)
    
    # Apply the following preprocessing steps to two copies of the data
    raw_for_ica = bandpass(l_freq = 1., h_freq = 1000)
    raw = bandpass(l_freq = 30, h_freq = 270)
    
    raw_for_ica = create_eogs(raw_for_ica)
    raw = create_eogs(raw)
    
    epochs_for_ica = epoch(raw_for_ica)
    epochs = epoch(raw)
    
    ica = compute_ICA(epochs_for_ica) # run ICA on less aggressively filtered data
    epochs = apply_ICA(epochs_for_ica, epochs) # apply ICA on more aggressively filtered data
    

## Functions
#### Import data 

In [ ]:
def get_bids_path(sub, task, run)
    bids_path = BIDSPath(root = BIDS_ROOT,
                        subject = sub,
                        task = task,
                        datatype = 'eeg',
                        run = run)
    return bids_path

def import_bids_data(bids_path)
    raw = read_raw_bids(bids_path, verbose = False)
    raw = raw.pick_types(eeg = True)
    return raw

def set_electrode_positions(raw, montage_name, stim_channel)
    dig = mne.channels.make_standard_montage(montage_name)
    raw = raw.set_channel_types({stim_channel: 'stim'}) % abstract?
    raw = raw.set_montage(dig)
    return raw

def read_events(raw)    
    events, events_ids = mne.events_from_annotations(raw)
    return events, events_ids

#### Resampling and PREP

In [ ]:
def resample(fs, events) # Resample to a more manageable speed
    raw, events = raw.resample(fs, events = events)
    return raw, events

def run_PREP(raw, sub_idx, LOWPASS) # Run PREP pipeline (notch, exclude bad channels, and re-reference)
    raw.load_data()
    np.random.seed(sub_idx)

    lf = raw.info['line_freq']
    prep_params = {
        'ref_chs': 'eeg',
        'reref_chs': 'eeg',
        'line_freqs': np.arange(lf, LOWPASS, lf) if np.arange(lf, LOWPASS, lf).size > 0 else [lf]
    }
    prep = PrepPipeline(raw, prep_params, raw.get_montage(), ransac = False, random_state = sub_idx)
    prep.fit()

    raw = prep.raw_eeg # replace raw with cleaned version
    return raw

#### Apply the following preprocessing steps to two copies of the data
Split the data into two copies, one filtered more liberally for ICA so that high frequency noise can be detected, one band-pass filtered at the behaviorally relevant frequencies. All of the following preprocessing steps will be applied to each of the copies.

In [ ]:
def bandpass(raw, l_freq, h_freq)
    raw = raw.filter(l_freq = l_freq, h_freq = h_freq)
    return raw

def create_eogs(raw)
    raw = mne.set_bipolar_reference(raw, anode = 'Fp1', cathode = 'FT10', ch_name = 'eog1', drop_refs = False)
    raw = mne.set_bipolar_reference(raw, anode = 'Fp2', cathode = 'FT9', ch_name = 'eog2', drop_refs = False)
    raw = raw.set_channel_types({'eog1': 'eog', 'eog2': 'eog'})
    return raw

def epoch(raw)
    epochs = mne.Epochs(
        raw, 
        events, 
        tmin = -0.2, 
        tmax = 0.250, 
        baseline = None, # do NOT baseline correct the trials yet; we do that after ICA
        event_id = event_ids, # remember which epochs are associated with which condition
        preload = True # keep data in memory
    )
    return epochs

def compute_ICA(raw, epochs)
    ica = ICA(n_components = 15, random_state = 0)
    ica.fit(epochs, picks = 'eeg')
    return ica

def apply_ICA(epochs_for_ica, epochs)
    eog_indices, eog_scores = ice.find_bads_eog(epochs_for_ica, threshold = 1.96)
    ica.exclude = eog_indices
    ica.apply(epochs) # apply to aggressively filtered version of data
    return epochs

# # Create virtual EOG channels with electrodes dropped to the face
# raw_for_ica = mne.set_bipolar_reference(raw_for_ica, anode = 'Fp1', cathode = 'FT10', ch_name = 'eog1', drop_refs = False)
# raw_for_ica = mne.set_bipolar_reference(raw_for_ica, anode = 'Fp2', cathode = 'FT9', ch_name = 'eog2', drop_refs = False)
# raw_for_ica = raw_for_ica.set_channel_types({'eog1': 'eog', 'eog2': 'eog'})

# raw = mne.set_bipolar_reference(raw, anode = 'Fp1', cathode = 'FT10', ch_name = 'eog1', drop_refs = False)
# raw = mne.set_bipolar_reference(raw, anode = 'Fp2', cathode = 'FT9', ch_name = 'eog2', drop_refs = False)
# raw = raw.set_channel_types({'eog1': 'eog', 'eog2': 'eog'})

In [ ]:
# # Epoch
# epochs_for_ica = mne.Epochs(
#     raw_for_ica, 
#     events, 
#     tmin = -0.2, 
#     tmax = 0.250, 
#     baseline = None, # do NOT baseline correct the trials yet; we do that after ICA
#     event_id = event_ids, # remember which epochs are associated with which condition
#     preload = True # keep data in memory
# )

# epochs = mne.Epochs(
#     raw, 
#     events, 
#     tmin = -0.2, 
#     tmax = 0.250, 
#     baseline = None, # do NOT baseline correct the trials yet; we do that after ICA
#     event_id = event_ids, # remember which epochs are associated with which condition
#     preload = True # keep data in memory
# )

In [ ]:
# # Compute ICA components on a copy of the data
# ica = ICA(n_components = 15, random_state = 0)
# ica.fit(epochs_for_ica, picks = 'eeg')

# # Plot ICA components
# fig_ica = ica.plot_components()

In [ ]:
# # Exclude ICA components correlated with EOG
# eog_indices, eog_scores = ice.find_bads_eog(epochs_for_ica, threshold = 1.96)
# ica.exclude = eog_indices
# ica.apply(epochs) # apply to aggressively filtered version of data

# # Plot excluded ICA components
# if ica.exclude:
#     fig_ica_removed = ica.plot_components(ica.exclude)

#### Apply final preprocessing steps on original copy of data
Back to applying preprocessing on only one copy of the data. ICA is finished.

In [ ]:


# Baseline correction after ICA
epochs = epochs.pick_types(eeg = True)
epochs = epochs.apply_baseline((-0.2, 0.))

In [ ]:
# Reject bad trials with hard-coded threshold
epochs = epochs.drop_bad(reject = {'eeg': REJECT_THRES})

#### Plot results

In [ ]:
# Plot preprocessed data
fig_erp = epochs['50'].average().plot(spatial_colors = True)

In [ ]:
# Plot unprocessed data
epochs_unprocessed = mne.Epochs(
    raw_unprocessed, 
    events, 
    tmin = -0.2, 
    tmax = 0.8, 
    baseline = (-0.2, 0.0), # baseline correct to be fair
    event_id = event_ids,
    preload = True
)
bad_fig = epochs_unprocessed.average().plot(spatial_colors = True)

#### Save results and generate report

In [ ]:
sink = DataSink(DERIV_ROOT, 'preprocessing')

# Save cleaned data
fpath = sink.get_path(
                subject = sub,
                task = task, 
                desc = 'clean',
                suffix = 'epo', # this suffix is following MNE, not BIDS, naming conventions
                extension = 'fif.gz',
                )
epochs.save(fpath, overwrite = True)

In [ ]:
# Generate a report
report = mne.Report(verbose = True)
report.parse_folder(op.dirname(fpath), pattern = '*epo.fif.gz', render_bem = False)
report.add_figs_to_section(
    fig_erp, 
    captions = 'Average Evoked Response', 
    section = 'evoked'
)
if ica.exclude:
    report.add_figs_to_section(
        fig_ica_removed, 
        captions = 'Removed ICA Components', 
        section = 'ICA'
    ) 
bads = prep.noisy_channels_original
html_lines = []
for line in pformat(bads).splitlines():
    html_lines.append('<br/>%s' % line) 
html = '\n'.join(html_lines)
report.add_htmls_to_section(html, captions = 'Interpolated Channels', section = 'channels')
report.add_htmls_to_section('<br/>threshold: {:0.2f} microvolts</br>'.format(thres['eeg'] * 1e6), 
                            captions = 'Trial Rejection Criteria', section = 'rejection')
report.add_htmls_to_section(epochs.info._repr_html_(), captions = 'Info', section = 'info')
report.save(op.join(sink.deriv_root, 'sub-%s.html'%sub), overwrite = True)

In [ ]:
# Check data files are saved
from mne_bids import print_dir_tree
print_dir_tree(BIDS_ROOT)